<a href="https://colab.research.google.com/github/merriekay/CS167Code/blob/main/Day06_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 06 Notes: Missing Data and Normalization

## Missing Data Handling

In [ ]:
import pandas
import numpy
from google.colab import drive
drive.mount('/content/drive')

#import the data and take a look at it
#You'll have to change this file path to get this line to work
titanic = pandas.read_csv('/content/drive/MyDrive/CS167/datasets/titanic.csv')
titanic.isna() #this returns a boolean that is True if data is NaN

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
887,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False
889,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
titanic.isna().any() #will return a list of columns and whether or not they have missing data

survived       False
pclass         False
sex            False
age             True
sibsp          False
parch          False
fare           False
embarked        True
class          False
who            False
adult_male     False
deck            True
embark_town     True
alive          False
alone          False
dtype: bool

### dropna()

In [ ]:
print("before: ", titanic.shape)
clean_titanic = titanic.dropna()
print("after: ", clean_titanic.shape)

before:  (891, 15)
after:  (182, 15)


In [ ]:
print("before: ", titanic.shape)
titanic.dropna(inplace=True)
print("after: ", clean_titanic.shape)

before:  (891, 15)
after:  (182, 15)


### fillna()

In [ ]:
# let's reload the data so that we have missing data again.
titanic = pandas.read_csv('/content/drive/MyDrive/CS167/datasets/titanic.csv')

#and fill the missing data this time.

print("before: ", titanic['age'].isna().any())
age_mean = titanic['age'].mean()
titanic['age'].fillna(age_mean, inplace=True)
print("after: ", titanic['age'].isna().any())
titanic.head(7)

before:  True
after:  False


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
5,0,3,male,29.699118,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.000000,0,0,51.8625,S,First,man,True,E,Southampton,no,True


## Normalization Code

In [ ]:
titanic = pandas.read_csv('/content/drive/MyDrive/CS167/datasets/titanic.csv')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
#convert sex to numeric
titanic['sex'] = titanic['sex'].replace(to_replace='female', value=1)
titanic['sex'] = titanic['sex'].replace(to_replace='male', value=0)
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,0,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,0,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
#gender
#calculate mean and std
s_mean = titanic['sex'].mean()
s_std = titanic['sex'].std()

#replace column with each entry's z-score
titanic['sex'] = (titanic["sex"] - s_mean)/s_std
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,-0.737281,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1.354813,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1.354813,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1.354813,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,-0.737281,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
#age
#calculate mean and std
a_mean = titanic['age'].mean()
a_std = titanic['age'].std()

#replace column with each entry's z-score
titanic['age'] = (titanic["age"] - a_mean)/a_std
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,-0.737281,-0.530005,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1.354813,0.571430,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1.354813,-0.254646,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1.354813,0.364911,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,-0.737281,0.364911,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## kNN Functions:

In [ ]:
def knn(specimen, data, k):
  data['distance_to_new'] = numpy.sqrt(
    (specimen['petal length']-data['petal length'])**2
    +(specimen['sepal length']-data['sepal length'])**2
    +(specimen['petal width']-data['petal width'])**2
    +(specimen['sepal width']-data['sepal width'])**2)
  
  sorted_data = data.sort_values(['distance_to_new']) 
  return sorted_data.iloc[0:5]['species'].mode()[0]

In [ ]:
def w_knn(specimen, data, k):
  #calculate the distance
  data['distance_to_new'] = numpy.sqrt(
    (specimen['petal length']-data['petal length'])**2
    +(specimen['sepal length']-data['sepal length'])**2
    +(specimen['petal width']-data['petal width'])**2
    +(specimen['sepal width']-data['sepal width'])**2)
  
  #calculate the weights (remember, the weights are 1/d^2)
  data['weight'] = 1.0/(data['distance_to_new']**2)

  #find the closest k
  sorted_data = data.sort_values(['distance_to_new']) 

  print(sorted_data.iloc[0:k]) #take a look at the results

  #sum the weights and return the class of the larger sum. 
  print(sorted_data.iloc[0:k].groupby(['species'])['weight'].sum())
  return sorted_data.iloc[0:k].groupby(['species'])['weight'].sum().idxmax()


## Exercise 
Normalize the predictor columns of the iris dataset using the Z-score
- Note: you need a way to transform the new reading (the specimen) that you will make the prediction on so that the new one and the training data will all be on the same scale. How can you do that?

Repeat your k-NN prediction code with the normalized data.
- Does the value of k change the predictions?
Compare using k=3, k=5 on each method (normalized and non-normalized).


In [ ]:
iris = pandas.read_csv('/content/drive/MyDrive/CS167/datasets/irisData.csv')
iris.head()

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# write your code here!